In [ ]:
from google.colab import files
files.upload()

!pip install kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
#importing dog vs cat dataset
kaggle competitions download -c dogs-vs-cats
 
 #extracting the compressed data
 from zipfile import ZipFile

dataset = '/content/dogs-vs-cats.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The file is extracted')

  import os
#counting the number of filesin train folder
path, dirs, files = next(os.walk('/content/train'))
file_count = len(files)
print('Number of images: ', file_count)

#printing the name of image
file_names = os.listdir('/content/train')
print(file_names)

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mping
from sklearn.model_selection import train_test_split
from google.colab.patches import cv2_imshow

#displaying images of cats and dogs

#display image of dogs
img = mping.imread('/content/train/dog.7256.jpg')
imgplt = plt.imshow(img)
plt.show()

#display image of cats
img = mping.imread('/content/train/cat.4113.jpg')
imgplt = plt.imshow(img)
plt.show()


file_name = os.listdir('/content/train')
for i in range(5):

 name = file_name[i]
 print(name[0:3])

 file_name = os.listdir('/content/train')
dog_count = 0
cat_count = 0

for img_file in file_names:
  name = img_file[0:3]

  if name == 'dog':
    dog_count += 1
  else:
    cat_count += 1
print('Numbers of dog images = ', dog_count)
print('Numbers of cat images = ', cat_count)

#resizing all images
os.mkdir('/content/image resized')
original_folder = '/content/train/'
resized_folder = '/content/image resized/'

for i in range(2000):

  filename = os.listdir(original_folder)[i]
  img_path = original_folder+filename

  img = Image.open(img_path)
  img = img.resize((224, 224))
  img = img.convert('RGB')

  newImgPath = resized_folder + filename
  img.save(newImgPath)

#display resized image of dogs
img = mping.imread('/content/image resized/dog.12315.jpg')
imgplt = plt.imshow(img)
plt.show()

#display image of cats
img = mping.imread('/content/image resized/cat.4113.jpg')
imgplt = plt.imshow(img)
plt.show()

#creating label for resized image of dogs and cats
#creating for loop to assign labels


filenames = os.listdir('/content/image resized/')

labels = []
for i in range(2000):

  file_name = filenames[i]
  label = file_name[0:3]

  if label == 'dog':
    labels.append(1)

  else:
    labels.append(0)

    print(filenames[0:5])
print(len(filenames))


print(labels[0:5])
print(len(labels))


#counting the images of dogs and cats out of 2000 images
values, counts = np.unique(labels, return_counts=True)
print(values)
print(counts)


#converting all resized images into numpy arrays
import cv2
import glob

image_directory = '/content/image resized/'
image_extension = ['png', 'jpg']

files = []

[files.extend(glob.glob(image_directory + '*.' + e)) for e in image_extension]

dog_cat_images = np.asarray([cv2.imread(file) for file in files])
print(dog_cat_images )


type(dog_cat_images)


print(dog_cat_images.shape)

X = dog_cat_images
Y = np.asarray(labels)

#TRAIN TEST SPLIT
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

print(X.shape, X_train.shape, X_test.shape)

#training images  test images


#scaling the data
X_train_scaled = X_train/225X_test_scaled = X_test/225

print(X_train_scaled)


#BUILDING THE NEURAL NETWORK

import tensorflow as tf
import tensorflow_hub as hub
from google.colab.patches import cv2_imshow  # Only needed in Google Colab

mobilenet_model = 'https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4'

pretrained_model = hub.KerasLayer(mobilenet_model, input_shape=(224, 224, 3), trainable=False

num_of_classes = 2

model = tf.keras.Sequential([
    tf.keras.layers.Lambda(lambda x: pretrained_model(x)),
    tf.keras.layers.Dense(num_of_classes)

])

model.summary()

model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['acc']
)
model.fit(X_train_scaled, Y_train, epochs=5)
score, acc = model.evaluate(X_test_scaled, Y_test)
print('Test Loss', score)
print('Test accuracy =', acc)

#PREDICTIVE SYSTEM
input_image_path = input('Path of image to be predicted')

input_image = cv2.imread(input_image_path)

cv2_imshow(input_image)

input_image_resize = cv2.resize(input_image, (224, 224))

input_image_scaled = input_image_resize/225.0

image_reshaped = np.reshape(input_image_scaled, [1, 224, 224, 3])


input_prediction = model.predict(image_reshaped)

print(input_prediction)

input_pred_label = np.argmax(input_prediction)

print(input_pred_label)

if input_pred_label == 0:
    print('The image shows a cat')

else:
    print('The image shows a dog')



    input_image_path = input('Path of image to be predicted')

input_image = cv2.imread(input_image_path)

cv2_imshow(input_image)

input_image_resize = cv2.resize(input_image, (224, 224))

input_image_scaled = input_image_resize/225.0

image_reshaped = np.reshape(input_image_scaled, [1, 224, 224, 3])


input_prediction = model.predict(image_reshaped)

print(input_prediction)

input_pred_label = np.argmax(input_prediction)

print(input_pred_label)

if input_pred_label == 0:
    print('The image shows a cat')

else:
    print('The image shows a dog')
